In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from config import *
from color_map import apply_colormap
from path import get_mask_prs_path, SCS_PATH, GTS_PATH
from data import VOC2012SegDataset

In [3]:
from PIL import Image
from functools import partial

import torch
import torchvision
from torchvision.models import segmentation as segmodels
from torchvision.transforms.functional import to_pil_image
from torchvision.transforms._presets import SemanticSegmentation

# Model

In [4]:
print(torch.hub.get_dir())
print(CONFIG["device"])

/home/olivieri/exp/data/torch_weights
cuda


In [5]:
BY_MODEL = "LRASPP_MobileNet_V3"
SPLIT_BY = "non-splitted"

mask_prs_path = get_mask_prs_path(BY_MODEL)
mask_prs_path

PosixPath('/home/olivieri/exp/data/prompts_data/by_model/LRASPP_MobileNet_V3/_mask_prs_')

In [ ]:
model = segmodels.lraspp_mobilenet_v3_large(weights=None, weights_backbone=None).to(CONFIG["device"])
model.load_state_dict(torch.load(Path(CONFIG['seg']['pretrained_weights_root_path']) / ("lraspp_mobilenet_v3_large-full-pt" + ".pth")))
model.eval()

preprocess = partial(SemanticSegmentation, resize_size=CONFIG['seg']['image_size'])() # same as default transforms, but with custom resizing.

# Inference

In [7]:
seg_dataset = VOC2012SegDataset(
    root_path=Path("/home/olivieri/exp/data/VOCdevkit"),
    split='train',
    resize_size=520,
    center_crop=True,
    with_unlabelled=False,
    mask_prs_path=None
)

In [8]:
idxs = list(range(0, 8))

scs_list, gts_list = seg_dataset[idxs]

In [9]:
scs = torch.stack(scs_list, dim=0)
gts = torch.stack(gts_list, dim=0)
scs.shape, gts.shape

(torch.Size([8, 3, 520, 520]), torch.Size([8, 1, 520, 520]))

In [ ]:
x = preprocess(scs)
logits = model(x)["out"]
prs = logits.argmax(dim=1, keepdim=True).float()
assert scs.shape[2:] == gts.shape[2:] == prs.shape[2:]
prs.shape

torch.Size([8, 1, 520, 520])

In [11]:
image_UIDs = seg_dataset.get_image_UIDs()

In [ ]:
for i, idx in enumerate(idxs):
    pr_col = apply_colormap([prs[i]], seg_dataset.get_color_map_dict(with_unlabelled=False))[0]
    # gt_col = apply_colormap([gts[i]], seg_dataset.get_color_map_dict(with_void=False))[0]
    pr_col_img = to_pil_image(pr_col)
    # gt_col_img = to_pil_image(gt_col)
    # sc_img = to_pil_image(scs[i])

    # pr_ovr = Image.blend(sc_img, pr_col_img, 0.8)
    # gt_ovr = Image.blend(sc_img, gt_col_img, 0.8)
    
    # concat_imgs = Image.new('RGB', (pr_ovr.width + gt_ovr.width, pr_ovr.height))
    # concat_imgs.paste(pr_ovr, (0, 0))
    # concat_imgs.paste(gt_ovr, (pr_ovr.width, 0))

    # torchvision.utils.save_image(prs[i]/255., f'/home/olivieri/exp/data/prompts_data/by_model/LRASPP_MobileNet_V3/_mask_prs_/mask_pr_{idx}.png')